# Deep Agent with Runloop Backend

This notebook demonstrates how to create a deep agent that operates on files in a Runloop devbox.

## Setup: Import dependencies

In [2]:
import os
from runloop_api_client import Runloop
from deepagents.graph import create_deep_agent
from deepagents.integrations.runloop import RunloopBackend

In [3]:
import os

from runloop_api_client import Runloop

client = Runloop(
    bearer_token=os.environ.get("RUNLOOP_API_KEY"),  # This is the default and can be omitted
)

devbox_view = client.devboxes.create()
print(devbox_view.id)

dbx_31Wy37EKz7tur7WyBxD0L


In [4]:
backend = RunloopBackend(devbox_id=devbox_view.id, client=client)

In [5]:
# Create the deep agent with the Runloop backend
agent = create_deep_agent(
    backend=backend,
    system_prompt="You are a helpful coding assistant that operates on files in a remote devbox.",
)

print("Agent created with Runloop backend!")

Agent created with Runloop backend!


In [6]:
agent.invoke({
    "messages": [
        { 
            "role": "user",
            "content": "could you list the files you have access to (not using the shell)"
        }
    ]}
)

{'messages': [HumanMessage(content='could you list the files you have access to (not using the shell)', additional_kwargs={}, response_metadata={}, id='af9024cc-2f43-4189-adb4-df4002ba16e8'),
  AIMessage(content=[{'text': "I'll list the files in the root directory for you using the `ls` tool.", 'type': 'text'}, {'id': 'toolu_01F5JG2vEa3MfUkkVJDECfar', 'input': {'path': '/'}, 'name': 'ls', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01AJgwWnMfcuFRsowUgKymfd', 'model': 'claude-sonnet-4-5-20250929', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 6021}, 'cache_creation_input_tokens': 6021, 'cache_read_input_tokens': 0, 'input_tokens': 3, 'output_tokens': 70, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-sonnet-4-5-20250929', 'model_provider': 'anthropic'}, id='lc_run--40e0b169-0f0e-4454-8d27-54ad0af8202b-0', tool_calls=[{'name': 'ls', 'arg

In [8]:
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())
    }
}

# Example: Ask the agent to create a file
result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "create a file called foo.md with a short 3 sentence poem'"
    }]
}, config=config)

# Print the agent's response
print(result["messages"][-1].content_blocks)

[{'type': 'text', 'text': "Perfect! I've created the file `foo.md` in `/home/user/` with a short 3 sentence poem about nature. The poem describes the morning sun, a gentle breeze, and nature's peaceful symphony."}]


In [13]:
print(backend.execute('ls -lsh').output)
print(backend.execute('cat foo.md').output)

total 4.0K
4.0K -rw-r--r-- 1 user user 152 Nov  6 17:58 foo.md

The morning sun breaks through the trees.
A gentle breeze whispers secrets to the leaves.
Nature's symphony plays on, bringing peace to all who listen.



## With GitHub code mounts

Create an agent that operates on a cloned GitHub repository.

In [3]:
# Create provider with GitHub code mount
provider_with_repo = RunloopProvider(
    api_key=os.environ.get("RUNLOOP_API_KEY"),
    create_params={
        "code_mounts": [
            {
                "repo_name": "personal-site-generator",  # Replace with your repo
                "repo_owner": "eyurtsev",  # Replace with your username
                "token": os.environ.get("GITHUB_TOKEN"),
            }
        ],
    }
)

# The None Here doesn't make much sense
backend = provider.backend_factory(None)

# Create agent
agent = create_deep_agent(
    backend=backend,
    system_prompt="You are a helpful coding assistant. The repository is mounted in the devbox.",
)

print("Agent created with GitHub repository mounted!")

Agent created with GitHub repository mounted!
